In [59]:
import pandas as pd
import re

In [131]:
import pandas as pd
import re

def expand_quarter_to_daily(df, quarter_col=None, use_index=False):
    """
    Expands rows in the DataFrame so that each valid quarter is expanded into daily rows
    covering the entire quarter, with the same values copied to each day.

    Expected quarter format: "year number kwartaal" (e.g., "2020 4e kwartaal" or "2020 4 kwartaal").
    Rows are skipped if:
      - The value is exactly "Bron: CBS"
      - The value is not in the expected quarter format (e.g., if it’s only a year or otherwise invalid).

    """
    def convert_quarter_string(s):
        s = s.strip()
        # Expected pattern: "YYYY <number>[optional letter] kwartaal"
        pattern = r"^(\d{4})\s+(\d+)[a-zA-Z]*\s+kwartaal$"
        m = re.fullmatch(pattern, s)
        if m:
            year = m.group(1)
            quarter = m.group(2)
            return pd.Period(f"{year}Q{quarter}", freq='Q')
        else:
            return None

    expanded_rows = []
    
    for idx, row in df.iterrows():
        # Extract the quarter string either from the index or the specified column.
        if use_index:
            quarter_str = str(idx).strip()
        else:
            if quarter_col is None:
                raise ValueError("quarter_col must be provided if use_index is False.")
            quarter_str = str(row[quarter_col]).strip()
        
        # Skip rows that should be dropped.
        if quarter_str == "Bron: CBS":
            continue
        
        period = convert_quarter_string(quarter_str)
        if period is None:
            continue  # Skip rows not matching the expected quarter format.
        
        start_date = period.start_time
        end_date = period.end_time
        
        # Create a daily date range for the quarter.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        # Duplicate the row for each day.
        row_expanded = pd.DataFrame([row] * len(daily_index), index=daily_index)
        row_expanded['date'] = daily_index
        
        expanded_rows.append(row_expanded)
    
    return pd.concat(expanded_rows) if expanded_rows else pd.DataFrame()



def expand_year_to_daily(df, year_col=None, use_index=False):
    """
    Expands rows in the DataFrame so that each valid year is expanded into daily rows
    covering the entire year, with the same values copied to each day.

    Expected year format: a four-digit year (e.g., "2020").
    Rows are skipped if:
      - The value is exactly "Bron: CBS"
      - The value is not a valid four-digit year.

    Returns:
        pd.DataFrame: A new DataFrame with a daily date index and expanded rows.
    """
    def convert_year_string(s):
        s = s.strip()
        # Expected: exactly a four-digit year.
        m = re.fullmatch(r"\d{4}", s)
        if m:
            return int(s)
        else:
            return None

    expanded_rows = []
    
    for idx, row in df.iterrows():
        # Extract the year string either from the index or from the specified column.
        if use_index:
            year_str = str(idx).strip()
        else:
            if year_col is None:
                raise ValueError("year_col must be provided if use_index is False.")
            year_str = str(row[year_col]).strip()
        
        # # Skip rows that should be dropped.
        # if year_str == "Bron: CBS":
        #     continue
        
        year_int = convert_year_string(year_str)
        if year_int is None:
            continue  # Skip rows that do not have a valid year.
        
        start_date = pd.Timestamp(year=year_int, month=1, day=1)
        end_date = pd.Timestamp(year=year_int, month=12, day=31)
        
        # Create a daily date range for the year.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        # Duplicate the row for each day.
# Convert the row to a dictionary and then create a DataFrame by replicating the dictionary values.
        row_dict = row.to_dict()
        row_expanded = pd.DataFrame({key: [value] * len(daily_index) for key, value in row_dict.items()},
                                    index=daily_index)
        row_expanded['date'] = daily_index
        
        expanded_rows.append(row_expanded)
    
    return pd.concat(expanded_rows) if expanded_rows else pd.DataFrame()

def expand_year_month_to_daily(df, year_month_col=None, use_index=False):
    """
    Expands rows in the DataFrame so that each valid year-month (e.g., "2020-03")
    is expanded into daily rows covering that entire month, with the same values
    repeated for each day.

    Parameters:
      df (pd.DataFrame): The original DataFrame.
      year_month_col (str, optional): Name of the column that contains the year-month
                                      information. Must be provided if use_index is False.
      use_index (bool): If True, the year-month info is taken from the DataFrame's index.

    Returns:
      pd.DataFrame: A new DataFrame with a daily date index (and an extra column 'date')
                    and expanded rows.
    """
    def convert_year_month_string(s):
        s = str(s).strip()
        try:
            # Try using Period conversion (expects something like "2020-03")
            return pd.Period(s, freq='M')
        except Exception:
            # If that fails, try to extract using regex (handles formats like "2020 3" too)
            m = re.match(r"(\d{4})[-\s]?(\d{1,2})", s)
            if m:
                year = int(m.group(1))
                month = int(m.group(2))
                return pd.Period(f"{year}-{month:02d}", freq='M')
            else:
                return None

    expanded_rows = []

    for idx, row in df.iterrows():
        # Extract the year-month string either from index or a specified column.
        if use_index:
            year_month_str = str(idx).strip()
        else:
            if year_month_col is None:
                raise ValueError("year_month_col must be provided if use_index is False")
            year_month_str = str(row[year_month_col]).strip()
        
        period = convert_year_month_string(year_month_str)
        if period is None:
            continue  # Skip rows that don't have a valid year-month
        
        # Determine start and end dates for the month.
        start_date = period.start_time  # first day of the month
        end_date = period.end_time      # last day of the month

        # Create a daily date range for that month.
        daily_index = pd.date_range(start=start_date, end=end_date, freq='D')
        
        # Duplicate the row for each day.
        # Instead of repeating the row (which can cause duplicate-index issues),
        # we create a dictionary with each column's value repeated.
        row_dict = row.to_dict()
        daily_df = pd.DataFrame({key: [value] * len(daily_index) for key, value in row_dict.items()},
                                index=daily_index)
        # Optionally add a 'date' column.
        daily_df['date'] = daily_index
        
        expanded_rows.append(daily_df)
    
    if expanded_rows:
        return pd.concat(expanded_rows)
    else:
        return pd.DataFrame()




In [114]:
LaborParticipation = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Arbeidsdeelname__kerncijfers_seizoengecorrigeerd_21022025_095354.csv',
sep=';', header=3)
LaborParticipation = LaborParticipation.drop([0])
LaborParticipation.head(5)
LaborParticipation_daily = expand_quarter_to_daily(LaborParticipation, 'Unnamed: 0')
LaborParticipation_daily.set_index('date',inplace=True)



In [77]:
ConsumerPriceIndex = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Consumentenprijzen__prijsindex_2015_100_21022025_095130.csv',sep=';', header=3)
ConsumerPriceIndex.drop([0])
ConsumerPriceIndex.head(5)
ConsumerPriceIndex_daily = expand_year_to_daily(ConsumerPriceIndex, 'Unnamed: 0')
ConsumerPriceIndex_daily.set_index('date')

,Unnamed: 0,CPI,CPI afgeleid,Jaarmutatie CPI,Jaarmutatie CPI afgeleid
date,,,,,
1996-01-01,1996,"68,94","73,25",NaN,NaN
1996-01-02,1996,"68,94","73,25",NaN,NaN
1996-01-03,1996,"68,94","73,25",NaN,NaN
1996-01-04,1996,"68,94","73,25",NaN,NaN
1996-01-05,1996,"68,94","73,25",NaN,NaN
...,...,...,...,...,...
2024-12-27,2024,"130,31","128,04","3,3","2,4"
2024-12-28,2024,"130,31","128,04","3,3","2,4"
2024-12-29,2024,"130,31","128,04","3,3","2,4"


In [ ]:
import pandas as pd



PopulationSize = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bevolking_op_1_januari_en_gemiddeld__geslacht__leeftijd_en_regio_23022025_161215.csv',sep=';',header=4)
PopulationSize = PopulationSize.drop(['Unnamed: 0'],axis=1)
PopulationSize.columns = PopulationSize.columns.str.replace(r'\d+', '', regex=True)
# Flatten the multi column setup
PopulationSize.iloc[1]
PopulationSize.columns += PopulationSize.iloc[1]

# Create daily intervals
PopulationSize_Daily = expand_year_to_daily(PopulationSize,'GeslachtLeeftijd')
PopulationSize_Daily.set_index('date',inplace=True)









Index(['GeslachtLeeftijd', 'Totaal mannen en vrouwenTotaal',
       'Totaal mannen en vrouwen.0 jaar', 'Totaal mannen en vrouwen.1 jaar',
       'Totaal mannen en vrouwen.2 jaar', 'Totaal mannen en vrouwen.3 jaar',
       'Totaal mannen en vrouwen.4 jaar', 'Totaal mannen en vrouwen.5 jaar',
       'Totaal mannen en vrouwen.6 jaar', 'Totaal mannen en vrouwen.7 jaar',
       ...
       'Totaal mannen en vrouwen.97 jaar', 'Totaal mannen en vrouwen.98 jaar',
       'Totaal mannen en vrouwen.99 jaar', 'Totaal mannen en vrouwen.100 jaar',
       'Totaal mannen en vrouwen.101 jaar',
       'Totaal mannen en vrouwen.102 jaar',
       'Totaal mannen en vrouwen.103 jaar',
       'Totaal mannen en vrouwen.104 jaar',
       'Totaal mannen en vrouwen.105 jaar of ouder',
       'Totaal mannen en vrouwen.Totaal'],
      dtype='object', length=110)

In [ ]:
Gdp = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Opbouw_binnenlands_product__bbp___nationale_rekeningen__1995_2022_21022025_094218.csv',sep=';', header=3).T
Gdp.columns = Gdp.iloc[0]
Gdp_daily = expand_year_to_daily(Gdp,use_index=True )
Gdp_daily.rename(columns={'Onderwerp':'date'}, inplace=True)


In [99]:
GdpProductionAndSpending = pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bbp__productie_en_bestedingen__kwartalen__mutaties__na__1995__2024_I_21022025_094045.csv',sep=';', header=3).T
GdpProductionAndSpending = GdpProductionAndSpending.reset_index(drop=True)
# set header of
GdpProductionAndSpending.columns = GdpProductionAndSpending.iloc[0]
GdpProductionAndSpending = GdpProductionAndSpending.drop([0,1])
GdpProductionAndSpending = GdpProductionAndSpending.drop('Bron: CBS', axis=1)   
GdpProductionAndSpending_daily = expand_quarter_to_daily(GdpProductionAndSpending, 'Onderwerp')

GdpProductionAndSpending_daily.set_index('date')

,Onderwerp,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Bruto binnenlands product,"Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Bbp, gecorrigeerd voor werkdageneffecten",Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Invoer van goederen en diensten|Totaal,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Invoer van goederen en diensten|Goederen,Opbouw bbp vanuit de finale bestedingen|Beschikbaar voor finale bestedingen|Invoer van goederen en diensten|Diensten,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Consumptieve bestedingen|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Consumptieve bestedingen|Huishoudens,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Consumptieve bestedingen|Overheid,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Bruto investeringen in vaste activa|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Bruto investeringen in vaste activa|Bedrijven en huishoudens,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Bruto investeringen in vaste activa|Overheid,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Nationale finale bestedingen|Verandering in voorraden,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Uitvoer van goederen en diensten|Totaal,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Uitvoer van goederen en diensten|Goederen,Opbouw bbp vanuit de finale bestedingen|Finale bestedingen|Uitvoer van goederen en diensten|Diensten
date,,,,,,,,,,,,,,,,,,,
1995-01-01,1995 1e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1995-01-02,1995 1e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1995-01-03,1995 1e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1995-01-04,1995 1e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
1995-01-05,1995 1e kwartaal,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,2020 4e kwartaal,"-0,1","2,2",.,"-3,2","-4,9","2,2","-0,1","2,2","1,8","1,5","2,3","2,8","3,1","1,6",.,"-2,5","-3,8","2,0"
2020-12-28,2020 4e kwartaal,"-0,1","2,2",.,"-3,2","-4,9","2,2","-0,1","2,2","1,8","1,5","2,3","2,8","3,1","1,6",.,"-2,5","-3,8","2,0"
2020-12-29,2020 4e kwartaal,"-0,1","2,2",.,"-3,2","-4,9","2,2","-0,1","2,2","1,8","1,5","2,3","2,8","3,1","1,6",.,"-2,5","-3,8","2,0"


In [129]:
Boughthomes =  pd.read_csv('/Users/ruben/Documents/GitHub/MsCThesisRubenCuriel2024/Data/CBS/Bestaande_koopwoningen__verkoopprijzen_prijsindex_2015_100_1995_2023_25022025_192415.csv',sep=';', header=3)

Boughthomes_daily = expand_year_month_to_daily(Boughthomes, year_month_col="Unnamed: 0")
Boughthomes_daily.set_index('date',inplace=True)
Boughthomes_daily

,Unnamed: 0,Prijsindex verkoopprijzen|Prijsindex bestaande koopwoningen,Prijsindex verkoopprijzen|Ontwikkeling t.o.v. voorgaande periode,Prijsindex verkoopprijzen|Ontwikkeling t.o.v. een jaar eerder,Aantal verkochte woningen|Aantal verkochte woningen,Aantal verkochte woningen|Ontwikkeling t.o.v. voorgaande periode,Aantal verkochte woningen|Ontwikkeling t.o.v. een jaar eerder,Gemiddelde verkoopprijs,Totale waarde verkoopprijzen
date,,,,,,,,,
1995-04-01,1995 april,"41,6","0,5",NaN,10626,"-10,2",NaN,89808,954
1995-04-02,1995 april,"41,6","0,5",NaN,10626,"-10,2",NaN,89808,954
1995-04-03,1995 april,"41,6","0,5",NaN,10626,"-10,2",NaN,89808,954
1995-04-04,1995 april,"41,6","0,5",NaN,10626,"-10,2",NaN,89808,954
1995-04-05,1995 april,"41,6","0,5",NaN,10626,"-10,2",NaN,89808,954
...,...,...,...,...,...,...,...,...,...
2023-01-27,2023,"180,5","-2,8","-2,8",182403,"-5,5","-5,5",416153,75908
2023-01-28,2023,"180,5","-2,8","-2,8",182403,"-5,5","-5,5",416153,75908
2023-01-29,2023,"180,5","-2,8","-2,8",182403,"-5,5","-5,5",416153,75908


In [130]:
# Reset index to turn the date index into a column
Gdp_daily_reset = Gdp_daily.reset_index()
GdpProductionAndSpending_daily_reset = GdpProductionAndSpending_daily.reset_index()
PopulationSize_Daily_reset = PopulationSize_Daily.reset_index()
ConsumerPriceIndex_daily_reset = ConsumerPriceIndex_daily.reset_index()
LaborParticipation_daily_reset = LaborParticipation_daily.reset_index()
Boughthomes_daily = Boughthomes_daily.reset_index()


# Merge the dataframes on the 'index' column (which holds the dates)
combined_df = Gdp_daily_reset.merge(GdpProductionAndSpending_daily_reset, on='date', how='outer') \
                         .merge(PopulationSize_Daily_reset, on='date', how='outer') \
                         .merge(ConsumerPriceIndex_daily_reset,on='date', how='outer')\
                         .merge(LaborParticipation_daily_reset,on='date', how='outer')\
                         .merge(Boughthomes_daily,on='date', how='outer')

# Optionally, set the 'index' column back as the index
combined_df.set_index('index', inplace=True)
combined_df.to_csv('test3.csv')

# Gdp_daily.join(GdpProductionAndSpending_daily,)

# df = combined_df[combined_df['date'] >= combined_df['01/01/1995']]

combined_df.index = pd.to_datetime(combined_df.index)

# Define the constant comparison date
cutoff_date = pd.Timestamp('1995-01-01')

# Filter rows where the index (date) is not earlier than the cutoff_date
filtered_df = combined_df[combined_df.index >= cutoff_date]

filtered_df.to_csv('test4.csv')

In [107]:
combined_df

,index_x,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Totaal,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Lonen,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Beloning van werknemers|Sociale premies t.l.v. werkgevers,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Saldo,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Belastingen op productie en invoer,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Belastingen en subsidies|Subsidies,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Bruto,Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Verbruik van vaste activa (-),Bbp vanuit de inkomensvorming|Waarde in werkelijke prijzen|Exploitatieoverschot|Netto,...,CPI,CPI afgeleid,Jaarmutatie CPI,Jaarmutatie CPI afgeleid,Unnamed: 0_y,Beroepsbevolking|werkloze beroepsbevolking|Werkloze beroepsbevolking,Beroepsbevolking|werkloze beroepsbevolking|Kortdurig,Beroepsbevolking|werkloze beroepsbevolking|Langdurig of onbekend,Beroepsbevolking|Werkloosheidspercentage,date_y
index,,,,,,,,,,,,,,,,,,,,,
NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"130,31","128,04","3,3","2,4",2024 4e kwartaal,374,314,60,"3,7",2024-12-27
2024-12-28,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"130,31","128,04","3,3","2,4",2024 4e kwartaal,374,314,60,"3,7",2024-12-28
2024-12-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"130,31","128,04","3,3","2,4",2024 4e kwartaal,374,314,60,"3,7",2024-12-29
